In [ ]:
# attrs: modeling, parsing + validation (same format)
# JSON (raw) --> Python(attrs) --> JSON
# serializasiton/deserialization
# cattrs: converters

In [ ]:
!pip install cattrs

In [ ]:
from attrs import field, define

import attrs
import cattrs
import uuid
import re
import json

from abc import ABC, abstractmethod

### attrs notes

`@attr.s(auto_attribs=True)` versus `@define`:

-  `@attr.s` is a decorator used to define a class with attributes specified within the class body.

- `auto_attribs=True`: This argument allows the use of type annotations to define attributes automatically. Without auto_attribs=True, you would need to use attr.ib() to define attributes.

- `@define` is newer and more explicit way introduced in attrs version 21.3.0 to define classes, meant to be more intuitive and clear. It also uses type annotations to define attributes, similar to `@attr.s(auto_attribs=True)`. It is seen as a more modern and straightforward way to define attrs classes.



If you define a class with a `attrs.field()` that lacks a type annotation, attrs will ignore other fields that have a type annotation, but are not defined using `attrs.field()`.

# Identifier Definition

### Identifier Class Proto 1

This is a naiive approach.

* reserved_names is a class variable.
* No dedicated getters or setters.
* All validation is performed in post init

In [ ]:
@define
class StixIdentifier:
    id: str = field()
    prefix: str = field(init=False)
    uuid: str = field(init=False)

    reserved_names = {
        "attack-pattern",
        "campaign",
        "course-of-action",
        "identity",
        "intrusion-set",
        "malware",
        "marking-definition",
        "note",
        "relationship",
        "tool",
        "x-mitre-asset",
        "x-mitre-collection",
        "x-mitre-data-source",
        "x-mitre-data-component",
        "x-mitre-matrix",
        "x-mitre-tactic",
    }

    def __attrs_post_init__(self):
        if "--" in self.id:
            self.prefix, self.uuid = self.id.split("--", 1)
        else:
            raise ValueError("Invalid id format. Expected format: ${prefix}--${uuid}")

        if self.prefix not in self.reserved_names:
            raise ValueError(
                f"Invalid prefix: {self.prefix}. Must be one of {self.reserved_names}"
            )

        try:
            uuid_obj = uuid.UUID(self.uuid, version=4)
        except ValueError:
            raise ValueError("Invalid UUID")

In [ ]:
# Example usage
try:
    random_uuid_string = str(uuid.uuid4())
    identifier = StixIdentifier(id=f"attack-pattern--{random_uuid_string}")
except ValueError as e:
    print(e)

The `StixIdentifier` will throw an exception if an ATT&CK ID string is not passed to the constructor:

In [ ]:
try:
  pass
  # Throws an error
  # id1 = StixIdentifier()
  # >>> TypeError: StixIdentifier.__init__() missing 1 required positional argument: 'id'
except ValueError as e:
  print(e)

In [ ]:
id1 = StixIdentifier("attack-pattern--00000000-0000-0000-0000-000000000000")
print(id1.uuid, type(id1.uuid))
print(id1.prefix, type(id1.prefix))

00000000-0000-0000-0000-000000000000 <class 'str'>
attack-pattern <class 'str'>


Let's validate that the `StixIdentifier` blocks the user from passing unsupported object types like `foobar`:

In [ ]:
try:
  badPrefix = StixIdentifier("foobar--00000000-0000-4000-8000-000000000000")
except ValueError as e:
  print(e)

Invalid prefix: foobar. Must be one of {'x-mitre-data-component', 'marking-definition', 'note', 'tool', 'relationship', 'intrusion-set', 'campaign', 'x-mitre-collection', 'x-mitre-matrix', 'identity', 'x-mitre-tactic', 'x-mitre-data-source', 'attack-pattern', 'course-of-action', 'malware', 'x-mitre-asset'}


In [ ]:
try:
  badUuid = StixIdentifier("attack-pattern--00000000-0000-0000-0000-0") # incorrect uuid format
except ValueError as e:
  print(e)

Invalid UUID


### Identifier Class Proto 2

Let's beef up the capabilities of the StixIdentifier class. It should support validating on the setters and also include a "freeze" mode where, when toggled on, all properties are incapability of being changed.

We'll also move the set of supported ATT&CK identifier prefixes outside of the class to conserve memory:

In [ ]:
import attr
import uuid
import json
import re

RESERVED_IDENTIFIER_PREFIXES = {
    "attack-pattern",
    "campaign",
    "course-of-action",
    "identity",
    "intrusion-set",
    "malware",
    "marking-definition",
    "note",
    "relationship",
    "tool",
    "x-mitre-asset",
    "x-mitre-collection",
    "x-mitre-data-source",
    "x-mitre-data-component",
    "x-mitre-matrix",
    "x-mitre-tactic",
}

@define
class StixIdentifier:
    id: str = field()
    _prefix: str = field(init=False)
    _uuid: str = field(init=False)
    frozen: bool = field(default=False, init=False)

    def __attrs_post_init__(self):
        stix_pattern = re.compile(
            r'^[a-z][a-z0-9-]+[a-z0-9]--[0-9a-fA-F]{8}-[0-9a-fA-F]{4}-[1-5][0-9a-fA-F]{3}-[89abAB][0-9a-fA-F]{3}-[0-9a-fA-F]{12}$'
        )
        if not stix_pattern.match(self.id):
            raise ValueError("ID does not match the required STIX 2.1 pattern")

        if "--" in self.id:
            self._prefix, self._uuid = self.id.split("--", 1)
        else:
            raise ValueError("Invalid id format. Expected format: ${prefix}--${uuid}")

        if self._prefix not in RESERVED_IDENTIFIER_PREFIXES:
            raise ValueError(
                f"Invalid prefix: {self._prefix}. Must be one of {RESERVED_IDENTIFIER_PREFIXES}"
            )

        try:
            uuid.UUID(self._uuid, version=4)
        except ValueError:
            raise ValueError(f"Invalid UUID: {self._uuid}")

    @property
    def prefix(self):
        return self._prefix

    @prefix.setter
    def prefix(self, value):
        if self.frozen:
            raise ValueError("Cannot modify prefix while object is frozen")
        if value not in RESERVED_IDENTIFIER_PREFIXES:
            raise ValueError(
                f"Invalid prefix: {value}. Must be one of {RESERVED_IDENTIFIER_PREFIXES}"
            )
        self._prefix = value
        self.id = f"{self._prefix}--{self._uuid}"

    @property
    def uuid(self):
        return self._uuid

    @uuid.setter
    def uuid(self, value):
        if self.frozen:
            raise ValueError("Cannot modify uuid while object is frozen")
        try:
            uuid.UUID(value, version=4)
        except ValueError:
            raise ValueError(f"Invalid UUID: {value}")
        self._uuid = value
        self.id = f"{self._prefix}--{self._uuid}"

    def freeze(self):
        """Freeze the object, preventing any further changes."""
        self.frozen = True

    def unfreeze(self):
        """Unfreeze the object, allowing changes."""
        self.frozen = False

    def to_json(self):
        """Return the ID as a JSON string."""
        return json.dumps(self.id, indent=4)

In [ ]:
# Example usage
try:
    random_uuid_string = str(uuid.uuid4())
    identifier = StixIdentifier(id=f"attack-pattern--{random_uuid_string}")
    print(identifier.to_json())

    # Modify properties while unfrozen
    identifier.prefix = "malware"
    identifier.uuid = str(uuid.uuid4())
    print(identifier.to_json())

    # Freeze the object
    identifier.freeze()

    # Attempting to modify properties should raise an error
    try:
        identifier.prefix = "tool"
    except ValueError as e:
        print(e)

    # Unfreeze the object
    identifier.unfreeze()

    # Modifying properties after unfreezing
    identifier.prefix = "tool"
    identifier.uuid = str(uuid.uuid4())
    print(identifier.to_json())

except ValueError as e:
    print(e)

"attack-pattern--1d28bec2-f2d3-4f4d-a420-b66d885459dd"
"malware--85cf8dc8-bed5-4871-b97e-7f3818aef89a"
Cannot modify prefix while object is frozen
"tool--5c43c706-1839-454a-af68-292881654041"


### Pydantic Approach

In [ ]:
!pip install pydantic

In [ ]:
from pydantic import BaseModel, validator, Field
from pydantic.error_wrappers import ValidationError
from uuid import UUID, uuid4
from typing import Literal, Set

# Set of reserved names
reserved_names: Set[str] = {
    "attack-pattern",
    "campaign",
    "course-of-action",
    "identity",
    "intrusion-set",
    "malware",
    "marking-definition",
    "note",
    "relationship",
    "tool",
    "x-mitre-asset",
    "x-mitre-collection",
    "x-mitre-data-source",
    "x-mitre-data-component",
    "x-mitre-matrix",
    "x-mitre-tactic",
}

class ATTACKIdentifier(BaseModel):
    uuid: UUID = Field(default_factory=uuid4)
    prefix: Literal[
        "attack-pattern",
        "campaign",
        "course-of-action",
        "identity",
        "intrusion-set",
        "malware",
        "marking-definition",
        "note",
        "relationship",
        "tool",
        "x-mitre-asset",
        "x-mitre-collection",
        "x-mitre-data-source",
        "x-mitre-data-component",
        "x-mitre-matrix",
        "x-mitre-tactic",
    ]
    frozen: bool = False

    @validator('uuid')
    def uuid_must_be_v4(cls, value):
        if value.version != 4:
            raise ValueError('uuid must be a valid UUIDv4')
        return value

    def __init__(self, attack_id: str, **data):
        prefix, uuid_str = attack_id.split('--')
        if prefix not in reserved_names:
            raise ValueError(f"Invalid prefix: {prefix}")
        data['prefix'] = prefix
        data['uuid'] = UUID(uuid_str)
        super().__init__(**data)

    def set_uuid(self, new_uuid: UUID):
        if self.frozen:
            raise AttributeError("Cannot modify uuid when frozen")
        if new_uuid.version != 4:
            raise ValueError('uuid must be a valid UUIDv4')
        self.uuid = new_uuid

    def set_prefix(self, new_prefix: str):
        if self.frozen:
            raise AttributeError("Cannot modify prefix when frozen")
        if new_prefix not in reserved_names:
            raise ValueError(f"Invalid prefix: {new_prefix}")
        self.prefix = new_prefix

    def freeze(self):
        self.frozen = True

    def unfreeze(self):
        self.frozen = False

/usr/local/lib/python3.10/dist-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')
<ipython-input-12-6836061fbb5b>:48: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.7/migration/
  @validator('uuid')


In [ ]:
# Example usage
try:
    attack_id = "attack-pattern--d9fb73cc-158b-4ddd-b22b-da39014df0f7"
    attack_identifier = ATTACKIdentifier(attack_id=attack_id)
    print(attack_identifier)

    # Attempting to modify uuid and prefix
    attack_identifier.set_uuid(uuid4())
    attack_identifier.set_prefix("malware")

    # Freeze the object
    attack_identifier.freeze()

    # These will raise exceptions
    # attack_identifier.set_uuid(uuid4())
    # attack_identifier.set_prefix("tool")

except ValidationError as e:
    print(e)

except ValueError as e:
    print(e)

except AttributeError as e:
    print(e)

uuid=UUID('d9fb73cc-158b-4ddd-b22b-da39014df0f7') prefix='attack-pattern' frozen=False


### Interface

Let's define an interface for this class:

In [ ]:
class AbstractStixIdentifier(ABC):
    @abstractmethod
    def to_json(self) -> str:
        pass

    @abstractmethod
    def freeze(self) -> None:
        pass

    @abstractmethod
    def unfreeze(self) -> None:
        pass

    @abstractmethod
    def prefix(self, value: str) -> None:
        pass

    @abstractmethod
    def uuid(self, value: str) -> None:
        pass

    @abstractmethod
    def test_not_implemented(self, value: str) -> None:
        pass


RESERVED_IDENTIFIER_PREFIXES = {
    "attack-pattern",
    "campaign",
    "course-of-action",
    "identity",
    "intrusion-set",
    "malware",
    "marking-definition",
    "note",
    "relationship",
    "tool",
    "x-mitre-asset",
    "x-mitre-collection",
    "x-mitre-data-source",
    "x-mitre-data-component",
    "x-mitre-matrix",
    "x-mitre-tactic",
}


@define
class StixIdentifier(AbstractStixIdentifier):
    id: str = attr.ib()
    _prefix: str = attr.ib(init=False)
    _uuid: str = attr.ib(init=False)
    frozen: bool = attr.ib(default=False, init=False)

    def __attrs_post_init__(self):
        stix_pattern = re.compile(
            r'^[a-z][a-z0-9-]+[a-z0-9]--[0-9a-fA-F]{8}-[0-9a-fA-F]{4}-[1-5][0-9a-fA-F]{3}-[89abAB][0-9a-fA-F]{3}-[0-9a-fA-F]{12}$'
        )
        if not stix_pattern.match(self.id):
            raise ValueError("ID does not match the required STIX 2.1 pattern")

        if "--" in self.id:
            self._prefix, self._uuid = self.id.split("--", 1)
        else:
            raise ValueError("Invalid id format. Expected format: ${prefix}--${uuid}")

        if self._prefix not in RESERVED_IDENTIFIER_PREFIXES:
            raise ValueError(
                f"Invalid prefix: {self._prefix}. Must be one of {RESERVED_IDENTIFIER_PREFIXES}"
            )

        try:
            uuid.UUID(self._uuid, version=4)
        except ValueError:
            raise ValueError(f"Invalid UUID: {self._uuid}")

    @property
    def prefix(self):
        return self._prefix

    @prefix.setter
    def prefix(self, value):
        if self.frozen:
            raise ValueError("Cannot modify prefix while object is frozen")
        if value not in RESERVED_IDENTIFIER_PREFIXES:
            raise ValueError(
                f"Invalid prefix: {value}. Must be one of {RESERVED_IDENTIFIER_PREFIXES}"
            )
        self._prefix = value
        self.id = f"{self._prefix}--{self._uuid}"

    @property
    def uuid(self):
        return self._uuid

    @uuid.setter
    def uuid(self, value):
        if self.frozen:
            raise ValueError("Cannot modify uuid while object is frozen")
        try:
            uuid.UUID(value, version=4)
        except ValueError:
            raise ValueError(f"Invalid UUID: {value}")
        self._uuid = value
        self.id = f"{self._prefix}--{self._uuid}"

    def freeze(self):
        """Freeze the object, preventing any further changes."""
        self.frozen = True

    def unfreeze(self):
        """Unfreeze the object, allowing changes."""
        self.frozen = False

    def to_json(self):
        """Return the ID as a JSON string."""
        return json.dumps(self.id, indent=4)

In [ ]:
# Example usage
try:
    random_uuid_string = str(uuid.uuid4())

    # Throws an exception
    # identifier = StixIdentifier(id=f"attack-pattern--{random_uuid_string}")
    # >> TypeError: Can't instantiate abstract class StixIdentifier with abstract method test_not_implemented

except ValueError as e:
    print(e)

### Identifier Class Proto 3

Let's fix the interface so it actually works:

In [ ]:
class AbstractStixIdentifier(ABC):

    @abstractmethod
    def id(self, value: str) -> None:
        pass

    @abstractmethod
    def prefix(self, value: str) -> None:
        pass

    @abstractmethod
    def uuid(self, value: str) -> None:
        pass

    @abstractmethod
    def to_json(self) -> str:
        pass


RESERVED_IDENTIFIER_PREFIXES = {
    "attack-pattern",
    "campaign",
    "course-of-action",
    "identity",
    "intrusion-set",
    "malware",
    "marking-definition",
    "note",
    "relationship",
    "tool",
    "x-mitre-asset",
    "x-mitre-collection",
    "x-mitre-data-source",
    "x-mitre-data-component",
    "x-mitre-matrix",
    "x-mitre-tactic",
}


@define
class StixIdentifier:
    id: str = field()
    prefix: str = field(init=False)
    uuid: str = field(init=False)

    def __attrs_post_init__(self):
        if "--" in self.id:
            self.prefix, self.uuid = self.id.split("--", 1)
            self.uuid = self.uuid.strip()
        else:
            raise ValueError("Invalid id format. Expected format: ${prefix}--${uuid}")

    @id.validator
    def _check_id(instance, attribute, value):
        stix_pattern = re.compile(
            r'^[a-z][a-z0-9-]+[a-z0-9]--[0-9a-fA-F]{8}-[0-9a-fA-F]{4}-[1-5][0-9a-fA-F]{3}-[89abAB][0-9a-fA-F]{3}-[0-9a-fA-F]{12}$'
        )
        if not stix_pattern.match(value):
            raise ValueError("ID does not match the required STIX 2.1 pattern")

        instance.prefix, instance.uuid = value.split("--", 1)
        instance.uuid = instance.uuid.strip()

    @prefix.validator
    def _check_prefix(instance, attribute, value):
        if value not in RESERVED_IDENTIFIER_PREFIXES:
            raise ValueError(f"Invalid prefix: {value}. Must be one of {RESERVED_IDENTIFIER_PREFIXES}")

    @uuid.validator
    def _check_uuid(instance, attribute, value):
        try:
            uuid.UUID(value, version=4)
        except ValueError:
            raise ValueError(f"Invalid UUID: {value}")

    def to_json(self) -> str:
        """Return the ID as a JSON string."""
        return json.dumps(self.id)

In [ ]:
# Example usage
try:
    random_uuid_string = str(uuid.uuid4())
    identifier = StixIdentifier(id=f"attack-pattern--{random_uuid_string}")
    print('identifier before: ', identifier.to_json())

    # immutable - the following will not work!
    identifier.prefix = 'x-mitre-matrix'
    identifier.uuid = str(uuid.uuid4())

    print('identifier after: ', identifier.to_json())

    print('as an object: ', identifier)

except ValueError as e:
    print(e)

identifier before:  "attack-pattern--2b853cbe-9ad2-4443-b141-a02f593786a7"
identifier after:  "attack-pattern--2b853cbe-9ad2-4443-b141-a02f593786a7"
as an object:  StixIdentifier(id='attack-pattern--2b853cbe-9ad2-4443-b141-a02f593786a7', prefix='x-mitre-matrix', uuid='39ae61f1-9323-4e35-a4df-a6bf5dcff510')


In [ ]:
@define
class TestObject:
  id: StixIdentifier = field()

# Example usage
try:
    random_uuid_string = str(uuid.uuid4())
    instance = TestObject(id=StixIdentifier(id=f"attack-pattern--{random_uuid_string}"))
    print(cattrs.unstructure(instance))

except ValueError as e:
    print(e)

{'id': {'id': 'attack-pattern--56c87593-5083-4eb0-9cd9-037c39c5cb37'}}


We'll need to address how to appropriately serialize/deserialize our Python classes to other formats like JSON and YAML using `cattrs` later!

# STIX Object Definition

### Handling Lists

We first need to define a `Converter` (from `cattrs`) to handle parsing/converting lists of objects to types:

In [ ]:
from collections.abc import Callable, Iterable
from typing import Any, TypeAlias, TypeVar


T = TypeVar("T")
ItemConv: TypeAlias = Callable[[Any], T]
ListConv: TypeAlias = Callable[[Iterable[Any]], list[T]]


def list_of(item_type: ItemConv[T]) -> ListConv[T]:
    def converter(iterable: Iterable[Any]) -> list[T]:
        if iterable is None:
            return None
        return [item if isinstance(item, item_type) else item_type(item) for item in iterable]
    return converter

Next we can define the `StixObject` class. It has an `object_marking`refs` property which is a list of `StixIdentifier` objects.

We want to use the `list_of` converter we just created to automate the serialization of each element in the `object_marking_refs` list:

In [ ]:
@define
class StixObject:
  list_of_stix_ids: list[StixIdentifier] = field(converter=list_of(StixIdentifier))

In [ ]:
@define
class StixObject():
    object_marking_refs: list[StixIdentifier] = field(converter=list_of(StixIdentifier))

    @object_marking_refs.validator
    def object_marking_refs(self, attribute, value):
        if not isinstance(value, list):
            raise ValueError("object_marking_refs must be a list")
        if not all(isinstance(id, (str, StixIdentifier)) for id in value):
            raise ValueError("All elements in object_marking_refs must be strings or StixIdentifier objects")

    def to_json(self) -> str:
        """Return the STIX object as a JSON string."""
        result = {
            "object_marking_refs": [ref.id for ref in self.object_marking_refs] if self.object_marking_refs else None,
        }
        return json.dumps(result, indent=4)

The following approach tries to initialize a `StixObject` instance using a list of `StixIdentifier` objects:

In [ ]:
# Example usage
try:
    random_uuid_string = str(uuid.uuid4())
    identifier = StixIdentifier(id=f"attack-pattern--{random_uuid_string}")

    # Pre-create the StixIdentifiers
    marking_ref_1 = StixIdentifier(id=f"marking-definition--{str(uuid.uuid4())}")
    marking_ref_2 = StixIdentifier(id=f"marking-definition--{str(uuid.uuid4())}")

    stix_object = StixObject(object_marking_refs=[marking_ref_1, marking_ref_2])
    print(stix_object.to_json())

except ValueError as e:
    print(e)

{
    "object_marking_refs": [
        "marking-definition--22992383-65e7-43f9-aa00-6cf695199c6b",
        "marking-definition--6309418d-5888-4ca1-8c99-250163df2585"
    ]
}


The next approach instead tries to initialize a `StixObject` instance using a list of identifier strings. The hope is that we can automatically serialize them too:

In [ ]:
# Example usage
try:
    random_uuid_string = str(uuid.uuid4())
    identifier = StixIdentifier(id=f"attack-pattern--{random_uuid_string}")

    # Just use raw strings for the identifiers
    object_marking_refs = [
        f"marking-definition--{str(uuid.uuid4())}",
        f"marking-definition--{str(uuid.uuid4())}"
    ]

    stix_object = StixObject(object_marking_refs=object_marking_refs)
    print(stix_object)
    # print(stix_object.to_json())
    # Throws:
    #   AttributeError: 'str' object has no attribute 'id'

except ValueError as e:
    print(e)

StixObject(object_marking_refs=['marking-definition--e1c35c4f-9eb9-4364-9d4f-cc729f40cda9', 'marking-definition--4d3d2a4c-d7f7-4456-b185-9ad59513e854'])


### Single Object Converter

In [ ]:
def stix_id_converter(id):
  if isinstance(id, str):
    return StixIdentifier(id=id)
  elif isinstance(id, StixIdentifier):
    return id
  else:
    raise ValueError("Invalid ID type")

@define
class Foo:
    a: StixIdentifier = field(converter=stix_id_converter)

f1 = Foo(a="attack-pattern--a5564927-0067-4bca-a12e-2252746a8063")
print(f1)
print(f1.a)

Foo(a=StixIdentifier(id='attack-pattern--a5564927-0067-4bca-a12e-2252746a8063', prefix='attack-pattern', uuid='a5564927-0067-4bca-a12e-2252746a8063'))
StixIdentifier(id='attack-pattern--a5564927-0067-4bca-a12e-2252746a8063', prefix='attack-pattern', uuid='a5564927-0067-4bca-a12e-2252746a8063')


### Deserializing to JSON: Dealing with Optional Properties

I don't love this approach, but we can strip null/None properties from a class when we serialize it using manualy logic in the `to_json` method:

In [ ]:
from typing import Optional
from attrs import asdict

@define
class C:
  id: Optional[str] = None

  def to_json(self) -> str:
      return json.dumps({k: v for k, v in asdict(self).items() if v is not None})

c = C()
c.to_json()

'{}'

Let's try using `cattrs` instead:

In [ ]:
from json import dumps
from cattrs.preconf.json import make_converter

In [ ]:
@define
class StixObject():
    id: StixIdentifier

# Example usage
stix_object = StixObject(id=StixIdentifier('attack-pattern--a5564927-0067-4bca-a12e-2252746a8063'))

c = make_converter()
dumps(c.unstructure(stix_object))

'{"id": {"id": "attack-pattern--a5564927-0067-4bca-a12e-2252746a8063"}}'

That doesn't look quite right. Let's see if we can modify the unstructuring logic for `StixIdentifier` objects and automate that when parent classes are unstructured:

In [ ]:
# Create a custom converter
def stix_identifier_converter(stix_id: StixIdentifier) -> str:
    return stix_id.id

c1 = make_converter()
c1.register_unstructure_hook(StixIdentifier, stix_identifier_converter)
dumps(c1.unstructure(stix_object))

'{"id": "attack-pattern--a5564927-0067-4bca-a12e-2252746a8063"}'

In [ ]:
import yaml


@define
class Child:
    id: StixIdentifier

    _converter = make_converter()
    _converter.register_unstructure_hook(StixIdentifier, stix_identifier_converter)

    def to_json(self):
        return self._emit('json')

    def to_yaml(self):
        return self._emit('yaml')

    def _emit(self, type: str = 'json'):
        if type == 'json':
            return self._converter.unstructure(self)
        elif type == 'yaml':
            return yaml.dump(self._converter.unstructure(self))
        else:
            raise ValueError("Invalid type")


@define
class Parent:
    foo: str
    child: Child

    def to_json(self):
        return dumps({
            'foo': self.foo,
            'child': self.child.to_json()
        })

    def to_yaml(self):
        return yaml.dump({
            'foo': self.foo,
            'child': self.child.to_yaml()
        })

# Example usage
stix_identifier = StixIdentifier('attack-pattern--a5564927-0067-4bca-a12e-2252746a8063')
child = Child(id=stix_identifier)
parent = Parent(foo='bar', child=child)

# Output JSON
print(parent.to_json())

{"foo": "bar", "child": {"id": "attack-pattern--a5564927-0067-4bca-a12e-2252746a8063"}}


## Getting Serious

First define types for STIX `type` and `spec_version`. Validate them by pinning their `value` attributes to enums:

In [ ]:
from typing import Optional
from enum import Enum


# Define enums for STIX type and spec version
class StixTypeEnum(str, Enum):
    ATTACK_PATTERN = "attack-pattern"
    CAMPAIGN = "campaign"
    COURSE_OF_ACTION = "course-of-action"
    IDENTITY = "identity"
    INDICATOR = "indicator"
    INTRUSION_SET = "intrusion-set"
    MALWARE = "malware"
    OBSERVED_DATA = "observed-data"
    REPORT = "report"
    THREAT_ACTOR = "threat-actor"
    TOOL = "tool"
    VULNERABILITY = "vulnerability"
    MARKING_DEF = "marking-definition"


class StixSpecVersionEnum(str, Enum):
    V2_1 = "2.1"
    V2_0 = "2.0"


@define
class StixType:
    value: StixTypeEnum

    def __str__(self):
        return self.value.value

@define
class StixSpecVersion:
    value: StixSpecVersionEnum

    def __str__(self):
        return self.value.value

Next, we'll redefine STIX identifiers to be a little bit simpler:

In [ ]:
# Define the regex pattern for STIX identifiers
stix_pattern = re.compile(
    r'^[a-z][a-z0-9-]+[a-z0-9]--[0-9a-fA-F]{8}-[0-9a-fA-F]{4}-[1-5][0-9a-fA-F]{3}-[89abAB][0-9a-fA-F]{3}-[0-9a-fA-F]{12}$'
)

def validate_stix_identifier(instance, attribute, value):
    if not stix_pattern.match(value):
        raise ValueError(f"{value} is not a valid STIX identifier")

    _prefix, _uuid = value.split('--', 1)

    if _prefix not in StixTypeEnum._value2member_map_:
        raise ValueError(f"{_prefix} is not a supported STIX identifier prefix")

    try:
        uuid.UUID(_uuid, version=4)
    except ValueError:
        raise ValueError(f"{_uuid} is not a valid UUID")

@define
class StixIdentifier:
    value: str = field(
        validator=validate_stix_identifier,
        metadata={'description': 'A valid STIX identifier'}
    )
    prefix: str = field(
        init=False,
        metadata={'description': 'The prefix of the STIX identifier'}
    )
    uuid: str = field(
        init=False,
        metadata={'description': 'The UUID of the STIX identifier'}
    )

    def __attrs_post_init__(self):
        self.prefix, self.uuid = self.value.split('--', 1)

    def __str__(self):
        return self.value

# Custom converter for StixIdentifier
def stix_identifier_to_str(stix_identifier: StixIdentifier) -> str:
    return stix_identifier.value


def str_to_stix_identifier(value: str) -> StixIdentifier:
    return StixIdentifier(value)

Let's also initialize a custom converter that will handle de/serializing our objects:

In [ ]:
from cattrs import Converter

attack_converter = Converter()

# Register converters for the STIXIdentifier

attack_converter.register_unstructure_hook(StixIdentifier, stix_identifier_to_str)
attack_converter.register_structure_hook(
    StixIdentifier, lambda d, _: str_to_stix_identifier(d)
)

We also need a type for `created_by_ref` but this attribute is just a STIX identifier, which we've already defined.

Let's try aliasing it:

In [ ]:
# Define StixCreatedByRef as a wrapper around StixIdentifier
class StixCreatedByRef(StixIdentifier):
    pass

We also need to define a type annotation for STIX timestamps:

In [ ]:
# Define the regex pattern for STIX timestamps
timestamp_pattern = re.compile(
    r"^[0-9]{4}-(0[1-9]|1[012])-(0[1-9]|[12][0-9]|3[01])T([01][0-9]|2[0-3]):([0-5][0-9]):([0-5][0-9]|60)(\.[0-9]+)?Z$"
)

def validate_stix_timestamp(instance, attribute, value):
    if not timestamp_pattern.match(value):
        raise ValueError(f"{value} is not a valid STIX timestamp")

@define
class StixTimestamp:
    value: str = field(
        validator=validate_stix_timestamp,
        metadata={'description': 'A valid STIX timestamp in RFC3339 format with a Z timezone'}
    )

    def __str__(self):
        return self.value

and a whole bunch of other types...

This is a naiive implementation of an external reference class.

In [ ]:
@define
class ExternalReference:
    source_name: str
    description: Optional[str] = None
    url: Optional[str] = None
    external_id: Optional[str] = None

In [ ]:
er1 = external_ref = ExternalReference(
    source_name="Example Source",
    description="An example description",
    url="http://example.com",
    external_id="EX123"
)

# Convert class to JSON
json1 = cattrs.unstructure(er1)
print(json1)

# Convert JSON to class
cls = cattrs.structure({'source_name': 'Example Source',
 'description': 'An example description',
 'url': 'http://example.com ',
 'external_id': 'EX123'}, ExternalReference)
print(cls)

{'source_name': 'Example Source', 'description': 'An example description', 'url': 'http://example.com', 'external_id': 'EX123'}
ExternalReference(source_name='Example Source', description='An example description', url='http://example.com ', external_id='EX123')


Let's demonstrate how we can unstructure one of these complex attrs classes recursively through a parent class:

In [ ]:
@define
class TestParent:
  child: StixIdentifier

tp1 = TestParent(StixIdentifier(f"attack-pattern--{uuid.uuid4()}"))

# If you unstructure the tp instance using a generic converter, cattrs won't know how to handle the StixIdentifier:
print(cattrs.unstructure(tp1))

# But the attack_converter DOES know how to unstructure the StixIdentifier!
print(attack_converter.unstructure(tp1))

{'child': {'value': 'attack-pattern--0e79cfb1-e6d2-480e-bb96-9a057ec747d6'}}
{'child': 'attack-pattern--0e79cfb1-e6d2-480e-bb96-9a057ec747d6'}


This is great and all, but we'd like to be able to deserialize these objects using a handy object method like `emit` or `to_json`, right?

In [ ]:
@define
class ExternalReference:
    source_name: str
    description: Optional[str] = None
    url: Optional[str] = None
    external_id: Optional[str] = None

    _converter = field(attack_converter,)

    def to_json(self):
        return json.dumps(self._converter.unstructure(self))

# Initialize the converter for ExternalReference class
ExternalReference._initialize_converter()

AttributeError: 'ExternalReference' object has no attribute 'to_json'

In [ ]:
from typing import List

def validate_confidence(instance, attribute, value):
    if value is not None and not (0 < value < 100):
        raise ValueError(f"{value} is not a valid confidence level. Must be between 1 and 99 inclusive.")

@define
class StixBoolean:
    value: bool

@define
class GranularMarking:
    marking_ref: StixIdentifier
    selectors: List[str]

    def to_json(self):
      return {
          "marking_ref": str(self.marking_ref),
          "selectors": self.selectors
      }

@define
class Extension:
    pass  # Define fields as necessary

Now let's bring it all together in the `StixObject` class:

In [ ]:
from typing import List, Dict, Union

@define
class StixObjectMixin:
    id: StixIdentifier = field(
        metadata={'description': 'The unique identifier for the STIX object'}
    )
    type: StixType = field(
        metadata={'description': 'The type of the STIX object'}
    )
    spec_version: StixSpecVersion = field(
        metadata={'description': 'The specification version of the STIX object'}
    )
    created: StixTimestamp = field(
        metadata={'description': 'The created property represents the time at which the first version of this object was created'}
    )
    modified: StixTimestamp = field(
        metadata={'description': 'The modified property represents the time that this particular version of the object was modified'}
    )
    created_by_ref: Optional[StixCreatedByRef] = field(
        default=None,
        metadata={'description': 'Reference to the creator of the STIX object'}
    )
    labels: Optional[List[str]] = field(
        default=None,
        metadata={'description': 'The labels property specifies a set of terms used to describe this object'}
    )
    revoked: Optional[StixBoolean] = field(
        default=None,
        metadata={'description': 'Indicates whether the object has been revoked'}
    )
    confidence: Optional[int] = field(
        default=None,
        validator=validate_confidence,
        metadata={'description': 'Identifies the confidence that the creator has in the correctness of their data'}
    )
    lang: Optional[str] = field(
        default=None,
        metadata={'description': 'Identifies the language of the text content in this object'}
    )
    external_references: Optional[List[ExternalReference]] = field(
        default=None,
        metadata={'description': 'A list of external references which refers to non-STIX information'}
    )
    object_marking_refs: Optional[List[StixIdentifier]] = field(
        default=None,
        metadata={'description': 'The list of marking-definition objects to be applied to this object'}
    )
    granular_markings: Optional[List[GranularMarking]] = field(
        default=None,
        metadata={'description': 'The set of granular markings that apply to this object'}
    )
    extensions: Optional[Dict[str, Union[Extension, dict]]] = field(
        default=None,
        metadata={'description': 'Specifies any extensions of the object, as a dictionary'}
    )

    def to_json(self):
        return {
            "id": str(self.id),
            "type": str(self.type),
            "spec_version": str(self.spec_version),
            "created_by_ref": str(self.created_by_ref) if self.created_by_ref else None,
            "labels": self.labels,
            "created": str(self.created),
            "modified": str(self.modified),
            "revoked": str(self.revoked.value) if self.revoked else None,
            "confidence": self.confidence,
            "lang": self.lang,
            # "external_references": [er.__dict__ for er in self.external_references] if self.external_references else None,
            "external_references": [er.to_json() for er in self.external_references] if self.external_references else None,
            "object_marking_refs": [str(omr) for omr in self.object_marking_refs] if self.object_marking_refs else None,
            "granular_markings": [gm.to_json() for gm in self.granular_markings] if self.granular_markings else None,
            "extensions": {k: (v.__dict__ if isinstance(v, Extension) else v) for k, v in self.extensions.items()} if self.extensions else None
        }

In [ ]:
# Example usage
stix_identifier = StixIdentifier(f"attack-pattern--{uuid.uuid4()}")
stix_type = StixType(StixTypeEnum.ATTACK_PATTERN)
stix_spec_version = StixSpecVersion(StixSpecVersionEnum.V2_1)
stix_created_by_ref = StixCreatedByRef(f"identity--{uuid.uuid4()}")
stix_timestamp = StixTimestamp("2023-05-06T12:34:56Z")
labels = ["label1", "label2"]
external_references = [ExternalReference(source_name="example-source")]
object_marking_refs = [StixIdentifier(f"marking-definition--{uuid.uuid4()}")]
granular_markings = [
    GranularMarking(marking_ref=StixIdentifier(f"marking-definition--{uuid.uuid4()}"), selectors=["selector1"])
]

stix_object = StixObjectMixin(
    id=stix_identifier,
    type=stix_type,
    spec_version=stix_spec_version,
    created_by_ref=stix_created_by_ref,
    labels=labels,
    created=stix_timestamp,
    modified=stix_timestamp,
    revoked=StixBoolean(True),
    confidence=80,
    lang="en",
    external_references=external_references,
    object_marking_refs=object_marking_refs,
    granular_markings=granular_markings,
    extensions={"extension-key": {"key": "value"}}
)

# Attack Object Definition

Now let's define the `AttackObject` class and use composition to bring in the `StixObject`:

In [ ]:
@define
class AttackObject:
    _stix: StixObjectMixin = field(
        metadata={'description': 'The STIX object associated with this attack'}
    )
    dummy_attribute: str = field(
        default="dummy",
        metadata={'description': 'A dummy attribute for demonstration purposes'}
    )

    def __getattr__(self, name):
        return getattr(self._stix, name)

    def to_json(self):
        data = self._stix.to_json()
        data["dummy_attribute"] = self.dummy_attribute
        return data

In [ ]:
# Example usage
stix_identifier = StixIdentifier(f"attack-pattern--{uuid.uuid4()}")
stix_type = StixType(StixTypeEnum.ATTACK_PATTERN)
stix_spec_version = StixSpecVersion(StixSpecVersionEnum.V2_1)
stix_created_by_ref = StixCreatedByRef(f"identity--{uuid.uuid4()}")
stix_timestamp = StixTimestamp("2023-05-06T12:34:56Z")
labels = ["label1", "label2"]

stix = StixObjectMixin(
    id=stix_identifier,
    type=stix_type,
    spec_version=stix_spec_version,
    created_by_ref=stix_created_by_ref,
    labels=labels,
    created=stix_timestamp,
    modified=stix_timestamp,
    revoked=StixBoolean(True),
    confidence=80,
    lang="en",
    external_references=external_references,
    object_marking_refs=object_marking_refs,
    granular_markings=granular_markings,
    extensions={"extension-key": {"key": "value"}}
)

attack_object = AttackObject(stix)

print(attack_object.to_json())
print(attack_object.id)  # Accessing StixObject attribute directly

{'id': 'attack-pattern--ffe39fb5-e8a4-4ac0-816c-bd6c8a17d1d0', 'type': 'attack-pattern', 'spec_version': '2.1', 'created_by_ref': 'identity--dbd810fa-cefa-468c-9a62-3ec8f2525094', 'labels': ['label1', 'label2'], 'created': '2023-05-06T12:34:56Z', 'modified': '2023-05-06T12:34:56Z', 'revoked': 'True', 'confidence': 80, 'lang': 'en', 'external_references': [{'source_name': 'example-source'}], 'object_marking_refs': ['marking-definition--4d6633ad-0d92-4dde-95d4-173b4aa35099'], 'granular_markings': [{'marking_ref': 'marking-definition--e83cabab-39f8-4a5d-9fda-80a4f782ebc5', 'selectors': ['selector1']}], 'extensions': {'extension-key': {'key': 'value'}}, 'dummy_attribute': 'dummy'}
attack-pattern--ffe39fb5-e8a4-4ac0-816c-bd6c8a17d1d0


Try accessing `id` through `attack_object` even though it's technically defined in `stix_object`:

In [ ]:
print(attack_object.id)
print(attack_object._stix.id)

attack-pattern--6de6debf-3d1a-4c8d-b58c-6efbf166103d
attack-pattern--6de6debf-3d1a-4c8d-b58c-6efbf166103d


In [ ]:
attack_object.to_json()

{'id': 'attack-pattern--6de6debf-3d1a-4c8d-b58c-6efbf166103d',
 'type': 'attack-pattern',
 'spec_version': '2.1',
 'created_by_ref': 'identity--41cba2ba-efbb-4846-8cb3-8278eb70d07f',
 'labels': ['label1', 'label2'],
 'created': '2023-05-06T12:34:56Z',
 'dummy_attribute': 'dummy'}